# End-to-end handwriting-to-LaTeX Demo

This nb demos the process of taking hand-written math and converts it to LaTeX using the top tools found during the research phase.

The writing and input is be done using tkinter, where writing will occur in a separate window. The math OCR will be done with MathPix's OCR API.

The first test of the full workflow produced a rendered equation from a handwritten input whith about 9/11 characacters correctly. There are two possible problems contributing to this inaccuracy. The first, is the canvas drawing is not the exact image being sent. A background PIL canvas is silently keeping track of the drawing using line objects native to PIL. These objects are not presented as smoothly as the tkinter counterparts see image 2 and image 3. Another source of error is inaccurate user input, ie, bad handwriting.  

Accuracy and hand-writing style aside, it does show we can enact an end-to-end solution for the product.

## Contents

1. Phase 1: tkinter implementation
    1. Simple Canvas Code
    2. Add A Save Button To The Canvas
    3. Pilot Canvas Code
    
2. Phase 2: MathPix OCR API implementation
3. Phase 3: Testing
    1. Test Case 0
    2. Test Case 1
    3. Test Case 2

In [3]:
import sys
import base64

from tkinter import Tk, Canvas, ttk, Button
from tkinter import constants as con

from PIL import ImageGrab, ImageTk, ImageDraw
import PIL

import string
import random
from IPython.display import Markdown as md

import requests
import json

import config as cfg

Values `app_id` and `app_key` are user-specific API keys necessary for access. They can be loaded from the configs folder.

In [11]:
app_id = cfg.math_pix_key["app_id"]  
app_key = cfg.math_pix_key["app_key"]

In [ ]:
def rnd_image_filename(N=7):
    """Returns a random string to follow a file name for uniqueness. We allow 7 of either 
    ascii uppercase characters or 10 numeric digits. This gives 7^36 = 2.6515x10^30 possibilities,
    which is how many letters and numbers in the English alphabet exsit. A choice of 7 nearly 
    guarantees no two files will be named the same even on consecutive runs. 

    :param N: Length of random character string to append to a file name, defaults to 7.
    :type N: int
    :return: String of filename appended with random characters.
    :rtype: string
    """
    
    filetag = ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))
    filename = "figures/canvas_img_" + filetag + ".png"
    return filename

## Phase 1: tkinter Implementation

The following code blocks weave tkinter's tutorial showcase of the code with a few lines from a code from this video (https://www.youtube.com/watch?v=OdDCsxfI8S0).
The code block below 'Pilot Canvas code' combines what happens in the code blocks below 'Simple Canvas code' and 'Canvas w Save Button'.

###  Simple Canvas Code

This code block is from the simple sketchpad code from tkinter's documentation found at https://tkdocs.com/tutorial/canvas.html.

In [12]:
def save_posn(event):
    """Saves positional coordinates of object event.

    :param event: Recorded mouse-click events for drawing.
    :type event: class tkinter.Event
    """
    global lastx, lasty
    lastx, lasty = event.x, event.y

def add_line(event):
    """Adds a line connection previous location of event to current event location.
    Event here is where the mouse was and is located. 

    :param event: Recorded mouse-click events for drawing.
    :type event: class tkinter.Event
    """
    canvas.create_line((lastx, lasty, event.x, event.y))
    savePosn(event)

root = Tk()
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

canvas = Canvas(root, bg="white")
canvas.pack()

canvas.grid(column=0, row=0, sticky=(con.N, con.W, con.E ,con.S))
canvas.bind("<Button-1>", save_posn)
canvas.bind("<B1-Motion>", add_line)

root.mainloop(0)

### Add A Save Button To The Canvas

This code is from a YouTube video showing how to save a drawing from a canvas. 
The actual output image is not a good representation of how we want our tool to work. 
This code is modifed to use the **rnd_image_filename** function.

In [ ]:
def save():
    """Saves a canvas file after appending with string of 
    random characters for uniquness.
    """
    filename = rnd_image_filename()
    canvas_image.save(filename)

def paint(event):
    """Places a dot at each mouse location corresponding to the click event. 
    On the draw object, adds a line connection nearby locations of where the 
    event occurs. This latter image is saved and represents the image drawn
    on the tkinter canvas by the user.

    :param event: Recorded mouse-click events for drawing.
    :type event: class tkinter.Event
    """
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    cv.create_oval(x1, y1, x2, y2, fill="black", width=2)
    draw.line([x1, y1, x2, y2], fill="black", width=2)
    
width = 600
height = 400
center = height//2
white = (255, 255, 255)
background = "white"
green = (0,128,0)

root = Tk()

# Instantiate the tkinter canvas to draw on. 
cv = Canvas(root, width=width, height=height, bg=background)
cv.pack()

# PIL create an empty image and draw object to draw on memory only.  It is not visible.
canvas_image = PIL.Image.new("RGB", (width, height), white)
draw = ImageDraw.Draw(canvas_image)

cv.pack(expand=True, fill="both")
cv.bind("<B1-Motion>", paint)

button=Button(text="save", command=save)
button.pack()

root.mainloop()

### Pilot Canvas Code

Combine elements from prior code to create a canvas w a save button with smooth line drawing.

In [22]:
def pilot_canvas(width=800, height=600, linewidth = 3, linecolor="BLACK"):
    """Opens a canvas widget using tkinter that allows a user to save their work.
    
    :param width: Canvas width, defaults to 800.
    :type width: int, optional
     
    :param height: Canvas height, defaults to 600.
    :type height: int, optional
        
    :param linewidth: Canvas marker linewidth, defaults to 3.
    :type linewidth: int, optional
     
    :param linecolor: Canvas marker linecolor, defaults to black.
    :type linecolor: str, optional
    
    :return: Dictionary of filename where canvas was saved.
    :rtype: dict
    """
    
    def save(N=5):
        """Saves a canvas file after appending with string of N 
         random characters for uniquness.
    
        :param N: Number of random characters to append, defaults to 5.
        :type N: int
        """
        
        filename["name"] = rnd_image_filename(N=N)
        canvas_image.save(filename["name"])
        print("File was saved as: ", filename["name"] )

    def save_posn(event):
        """Saves positional coordinates of object event.
        
        :param event: Recorded mouse-click events for drawing.
        :type event: class tkinter.Event
        """

        global lastx, lasty
        lastx, lasty = event.x, event.y

    def add_line(event):
        """Adds a line connection previous location of event to current event location.
        Event here is where the mouse was and is located. Also draws the same line
        on a PIL image which represents the image drawn on the tkinter canvas by the 
        user. This is the image that will actually be saved and used by the OCR.
        
        :param event: Recorded mouse-click events for drawing.
        :type event: class tkinter.Event
        """
        # This canvas call is what the user sees on the screen. 
        canvas.create_line((lastx, lasty, event.x, event.y),
                           smooth=True, width=linewidth, fill=linecolor)
        
        # The draw call is in the background (invisible) 
        # capturing what will actually get converted to an image.
        draw.line([lastx, lasty, event.x, event.y], fill=linecolor, width=linewidth, joint='curve')
        save_posn(event)
        
    offset = (linewidth)/2
    filename = {}

    root = Tk()

    # Instantiate the tkinter canvas to draw on. 
    canvas = Canvas(root, bg="white", width=width, height=height)
    canvas.pack()
    
    # PIL create an empty image and draw object to draw on memory only.  It is not visible.
    canvas_image = PIL.Image.new("RGB", (width, height), (255, 255, 255))
    draw = ImageDraw.Draw(canvas_image)

    canvas.pack(expand=True, fill="both")
    canvas.bind("<Button-1>", save_posn)
    canvas.bind("<B1-Motion>", add_line)
    
    button = Button(text="Save Image", command=lambda: save(N=7))
    button.pack()

    root.mainloop()

    return filename

## Phase 2: MathPix OCR API implementation

This code block provides a wrapper function for submitting an API request to MathPix.
The `app_id` and `app_key` from above are necessary for this function to work. This code originally comes from teh **top_ocr_tools_mathpix_snip** nb.

In [24]:
def ocr_request(filename):
    """Sends an API request to MathPix API to be handled by the OCR .
    
    :param filename: Dictionary with the filename of the image to be sent to the MathPix API for a LaTeX prediction.
    :type filename: dict
 
    :return: Latex-formatted string.
    :rtype: str
    """
    
    dict_request={
            "src": "data:image/png",
            "formats": ["text", "data", "html"],
            "data_options": {
            "include_asciimath": True,
            "include_latex": True
            }
        }

    # Put desired filename from earlier.
    file_path = filename["name"]
    image_uri = "data:image/png;base64," + base64.b64encode(open(file_path, "rb").read()).decode()

    # Send the request.
    r = requests.post("https://api.mathpix.com/v3/text",
                      data=json.dumps({'src': image_uri}),
                      headers={"app_id": app_id, 
                               "app_key": app_key,
                               "Content-type": "application/json"})

    print(json.dumps(json.loads(r.text), indent=4, sort_keys=True))
    
    json_return = json.loads(r.text)
    latex_return = json_return.get("latex_styled")
    
    print(latex_return)
    print()
    
    return latex_return

## Phase 3: Testing

The following provides several test cases. Some have pre-populated calls that already include the return from the API so we can quickly compare it with the input canvas drawing.

## Test Case 0:

Make up a random equation and try it out.
Return from random equation drawn on the canvas. The input call is omitted.


In [ ]:
returned = {
  "text": "\\( \\sum_{m}\\left(_{j m}^{2}+\\tan \\left(\\phi_{m}\\right)\\right. \\)",
  "confidence": 0.47379128643166557,
  "is_printed": False,
  "request_id": "3b08353e2e65e4a05c5d68a3061032db",
  "latex_styled": "\\sum_{m}\\left(_{j m}^{2}+\\tan \\left(\\phi_{m}\\right)\\right.",
  "is_handwritten": True,
  "confidence_rate": 0.47379128643166557,
  "auto_rotate_degrees": 0,
  "auto_rotate_confidence": 0.011462837151636762
}

# this is what was returned
returned_ltx = "\\sum_{m}\\left(_{j m}^{2}+\\tan \\left(\\phi_{m}\\right)\\right."
md("$ %s $"%(returned_ltx))

### Results

**PIL's Image**

<img src="figures/canvas_img_RB.png" alt="" title="" width="400" height="300" />

**Rendered LaTeX**

<img src="figures/ren_latex_RB.png" alt="" title="" width="400" height="300" />

**Original LaTeX**  
$\huge \sum_m y^2_m + \tan(\phi_m) $

We see that almost every character except the 'y' was converted correctly. This is in part due to bad hand-writing. Looking at how y is actually written, with a partial break at the stem, we can see how the OCR thought this was two seperate characters: one closely resembling a j and the second part of the curve resembing a parenthesis. Fortunately, a majority of the LaTeX was successfully converted, so a user could come back and fix this minor LaTeX issue.

## Test Case 1:

Take a closer look at how the drawing looks on the canvas vs the PIL's version of the drawing. The image on the right is saved and sent to the OCR. We see that PIL's image has more roughness to it, which may affect OCR accuracy. This is an issue that likely we will revisit in the coming development phases.

<p float="left">
  <img src="figures/img_6V3SBX4_cv.png" alt="" title="" width="400" height="300" />
  <img src="figures/img_6V3SBX4_pil.png" alt="" title="" width="400" height="300" /> 
</p>

## Test Case 2:

Hand-write and apply end-to-end solution to the following equation:

$$ \huge \frac{\partial c}{\partial t} = \nabla \cdot (D \nabla c) - \nabla \cdot (\mathbf{v} c) + R $$

The file 'figures/canvas_img_YFXS3KF.png' is the saved version of my hand-drawn image. You can draw your own and use that file instead buy passing the correct argument to `ocr_request`.

In [ ]:
# Uncomment this line below to call in the canvas to draw on.  
# We can also specify a specific file from a previous canvas session.
# filename = pilot_canvas()

# Specify a specific file to convert.
filename = {"name":'figures/canvas_img_LKNB0E0.png'}

In [ ]:
# Send the API request.
latex_return=ocr_request(filename)

# Print the returned request.
md("$ %s $"%(latex_return))

### Compare inputs and outputs to ideal

**PIL's Image**
<p float="left">
  <img src=figures/canvas_img_YFXS3KF.png width="300" />
</p>

**Rendered LaTeX Image**
<p float="left">
  <img src=figures/ren_latex_YFXS3KF.png width="350" />
</p>


**Original LaTeX**  
$ \LARGE \frac{\partial c}{\partial t} = \nabla \cdot (D \nabla c) - \nabla \cdot (\mathbf{v} c) + R $


### How did we do?

In test case 0, roughly 1/10 characters was incorrect, with 1 error not counted, since it's an extraneous character. In test case 2, 3/19 characters were incorrect, with the same mistake being made twice ('<' instead of '('). In test case 1, we did not translate the LaTeX but showed how the canvas drawing is being translated to a PIL image.

This leads us to think more about how we would evaluate a character-by-character measurement of accuracy. What if the translation introduces more characters than exist in the original? What if we have the correct character, but it's being placed incorrectly ($\partial c$ vs $\partial_c$)?

We can see just from test case 2 that there is a learning curve for how the user must adapt their handwriting to give the OCR the best shot at correctly converting their handwriting.